In [245]:
# Working on identifying which celestial objects land on the GFAs

from astropy.io import fits
from astropy.table import Table
import desimodel.io
import desimodel.footprint
%pylab

sweep = Table.read('sweep-050p030-060p035.fits')
sweep1 = Table.read('sweep-060p065-070p070.fits')

plot(sweep['RA'], sweep['DEC'], 'b.')
tiles = desimodel.io.load_tiles()
#print(tiles.dtype.names)
#print(tiles['TILEID'])

radec = desimodel.footprint.get_tile_radec(4)
#print(radec)
#print(tiles['RA'])
#print(tiles['DEC'])
for targets in sweep:
    if(targets['FLUX_R'] > 0):
        print(targets['FLUX_R'])
#sweep
sweep1

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


RELEASE,BRICKID,BRICKNAME,OBJID,TYPE,RA,DEC,RA_IVAR,DEC_IVAR,DCHISQ [5],EBV,FLUX_G,FLUX_R,FLUX_Z,FLUX_W1,FLUX_W2,FLUX_W3,FLUX_W4,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,FLUX_IVAR_W1,FLUX_IVAR_W2,FLUX_IVAR_W3,FLUX_IVAR_W4,MW_TRANSMISSION_G,MW_TRANSMISSION_R,MW_TRANSMISSION_Z,MW_TRANSMISSION_W1,MW_TRANSMISSION_W2,MW_TRANSMISSION_W3,MW_TRANSMISSION_W4,NOBS_G,NOBS_R,NOBS_Z,NOBS_W1,NOBS_W2,NOBS_W3,NOBS_W4,RCHISQ_G,RCHISQ_R,RCHISQ_Z,RCHISQ_W1,RCHISQ_W2,RCHISQ_W3,RCHISQ_W4,FRACFLUX_G,FRACFLUX_R,FRACFLUX_Z,FRACFLUX_W1,FRACFLUX_W2,FRACFLUX_W3,FRACFLUX_W4,FRACMASKED_G,FRACMASKED_R,FRACMASKED_Z,FRACIN_G,FRACIN_R,FRACIN_Z,ANYMASK_G,ANYMASK_R,ANYMASK_Z,ALLMASK_G,ALLMASK_R,ALLMASK_Z,WISEMASK_W1,WISEMASK_W2,PSFSIZE_G,PSFSIZE_R,PSFSIZE_Z,PSFDEPTH_G,PSFDEPTH_R,PSFDEPTH_Z,GALDEPTH_G,GALDEPTH_R,GALDEPTH_Z,WISE_COADD_ID,FRACDEV,FRACDEV_IVAR,SHAPEDEV_R,SHAPEDEV_R_IVAR,SHAPEDEV_E1,SHAPEDEV_E1_IVAR,SHAPEDEV_E2,SHAPEDEV_E2_IVAR,SHAPEEXP_R,SHAPEEXP_R_IVAR,SHAPEEXP_E1,SHAPEEXP_E1_IVAR,SHAPEEXP_E2,SHAPEEXP_E2_IVAR
int32,int32,str8,int32,str4,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,int16,int16,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,int16,int16,int16,uint8,uint8,float32,float32,float32,float32,float32,float32,float32,float32,float32,str8,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
4000,635401,0658p670,0,PSF,66.1454199468,66.9084831807,4.72097e+09,4.43343e+10,1696.91 .. 0.0,0.5057,0.0,89.9539,0.0,-3.52,-6.55881,-68.2792,-208.378,0.0,0.196921,0.0,2.415,0.540222,0.00139922,3.02673e-05,0.223806,0.364807,0.568904,0.917868,0.948729,0.988838,0.995771,0,1,0,101,100,16,16,0.0,4.08859,0.0,16.8816,2.33363,0.549687,0.146833,0.0,0.0,0.0,0.0798347,0.071718,0.684666,1.41097,0.0,0.889268,0.0,0.0,0.983889,0.0,0,3,0,0,3,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0668p666,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4000,635401,0658p670,1,DEV,66.1253740885,66.9083976993,2.33287e+08,3.8971e+11,4220.77 .. 3351.96,0.509157,0.0,2341.74,0.0,-6.83844,-12.82,-1053.54,-3619.38,0.0,0.00426048,0.0,0.0385718,0.00983925,3.44313e-05,1.14029e-06,0.221527,0.362301,0.566714,0.917331,0.948388,0.988762,0.995742,0,1,0,100,100,15,16,0.0,29.6654,0.0,1.54533,0.260997,0.228251,0.0608615,0.0,0.159622,0.0,4.45708,1.9864,0.124554,0.329664,0.0,0.615911,0.0,0.0,0.196702,0.0,0,4,0,0,4,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0668p666,1.0,1.0,146.407,146830.0,-0.998976,2.3774e+06,0.00477209,1.15825e+06,0.0,0.0,0.0,0.0,0.0,0.0
4000,635401,0658p670,2,SIMP,66.119496885,66.9087245425,5.4273e+08,1.01007e+10,377.292 .. 0.0,0.510193,0.0,158.429,0.0,-6.94743,-3.93703,2.16558,229.356,0.0,0.0140095,0.0,2.38048,0.52552,0.00135402,2.88385e-05,0.220849,0.361554,0.56606,0.91717,0.948286,0.988739,0.995733,0,1,0,101,101,15,16,0.0,2.67833,0.0,3.21104,0.893549,0.805723,0.235319,0.0,0.136904,0.0,0.0558047,0.208612,42.9648,2.19395,0.0,0.947699,0.0,0.0,0.826869,0.0,0,3,0,0,3,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0668p666,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.45,0.45,0.0,0.0,0.0,0.0
4000,635401,0658p670,3,EXP,65.8779512588,66.9094045879,4.38489e+07,1.46808e+11,12519.8 .. 11703.1,0.557375,0.0,4122.78,0.0,24.2406,-2.26082,-377.466,-366.082,0.0,0.00378761,0.0,0.144967,0.0384864,0.000147072,4.89567e-06,0.192061,0.329089,0.537041,0.909866,0.943641,0.987704,0.995339,0,0,0,106,107,17,17,0.0,6.7321,0.0,16.982,1.75464,0.477412,0.101501,0.0,0.177873,0.0,1.39863,5.16148,0.250226,2.0485,0.0,0.743376,0.0,0.0,0.316783,0.0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0668p666,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38.1567,77625.0,-0.878348,2.12995e+06,0.00094475,44255.0
4000,635401,0658p670,4,DEV,65.8843825997,66.9089025206,1.8

In [237]:
"""newra, newdec = xy2radec(8.37, -10.65, -138.345, -333.179)
8.927313423598427
-9.324956250231294"""

import desimodel.io
gfatable = desimodel.io.load_gfa()
gfatable
import desimodel.focalplane
tiles = desimodel.io.load_tiles()
#print(indices)
#indices = desimodel.footprint.find_tiles_over_point(tiles, 8.37, -10.65)
#[8254, 15072, 198, 2213, 4225, 6242, 8255, 10265, 12279]
#print(tiles[8254])
#tile index 8254 has tileid 23658

telra, teldec = desimodel.footprint.get_tile_radec(23658)
#print(telra, teldec)
ra, dec = desimodel.focalplane.xy2radec(7.11, -10.53, 116.279135121, -372.885546514)
#print(ra, dec)
#Certain gfa location is in ra, dec respectively 6.644525362152656 -9.055425745149217

In [238]:
def is_on_gfa(tileid, ra, dec):
    """
    Checks if a target is on any of the 10 GFAs given a tileid and an array of RA and Dec pointings. 
    Parameters
    ----------
    tileid: An integer representing the tile's unique ID which can retrieve its RA and Dec pointing
    ra: An array of RA values for locations in the sky
    dec: An array of declination values for locations in the sky
    """
    import desimodel.footprint
    import desimodel.io
    # If any calculated area is under the threshold area, it is mathematically impossible
    THRESHOLD_AREA = 469.7
    TOLERANCE = 100
    MIN_TOLERANCE = .001
    # The area boundary's value is the area of the gfa plus some tolerance. 
    AREA_BOUNDARY = 469.8037221 + MIN_TOLERANCE + TOLERANCE
    
    targetindices = []
    gfatable = desimodel.io.load_gfa()
    telra, teldec = desimodel.footprint.get_tile_radec(tileid)
    #print(telra, teldec)
    targetx, targety = desimodel.focalplane.radec2xy(telra, teldec, ra, dec)
    targetx = np.asarray(targetx)
    targety = np.asarray(targety)
    
    # Method to check if point is inside the rectangle
    for gfaid in range(0, 40, 4):
        # a1 through a4 are edge lengths of the rectangle formed by corners of the GFAs
        a1 = np.sqrt((gfatable['X'][gfaid] - gfatable['X'][gfaid + 1])**2
                    + (gfatable['Y'][gfaid] - gfatable['Y'][gfaid + 1])**2)
        a2 = np.sqrt((gfatable['X'][gfaid + 1] - gfatable['X'][gfaid + 2])**2
                    + (gfatable['Y'][gfaid + 1] - gfatable['Y'][gfaid + 2])**2)
        a3 = np.sqrt((gfatable['X'][gfaid + 2] - gfatable['X'][gfaid + 3])**2
                    + (gfatable['Y'][gfaid + 2] - gfatable['Y'][gfaid + 3])**2)
        a4 = np.sqrt((gfatable['X'][gfaid + 3] - gfatable['X'][gfaid])**2
                    + (gfatable['Y'][gfaid + 3] - gfatable['Y'][gfaid])**2)
        # b1 through b4 are the line segments from each corner to the target location
        b1 = np.sqrt((gfatable['X'][gfaid] - targetx)**2
                    + (gfatable['Y'][gfaid] - targety)**2)
        b2 = np.sqrt((gfatable['X'][gfaid + 1] - targetx)**2
                    + (gfatable['Y'][gfaid + 1] - targety)**2)
        b3 = np.sqrt((gfatable['X'][gfaid + 2] - targetx)**2
                    + (gfatable['Y'][gfaid + 2] - targety)**2)
        b4 = np.sqrt((gfatable['X'][gfaid + 3] - targetx)**2
                    + (gfatable['Y'][gfaid + 3] - targety)**2)
        # Calculating areas of triangles using Heron's Formula
        #gfaarea = a1 * a2
        #gfaarea2 = a2 * a3
        u1 = (a1 + b1 + b2) / 2.0
        u2 = (a2 + b2 + b3) / 2.0
        u3 = (a3 + b3 + b4) / 2.0
        u4 = (a4 + b4 + b1) / 2.0
        area1 = np.sqrt(u1 * (u1 - a1) * (u1 - b1) * (u1 - b2))
        area2 = np.sqrt(u2 * (u2 - a2) * (u2 - b2) * (u2 - b3))
        area3 = np.sqrt(u3 * (u3 - a3) * (u3 - b3) * (u3 - b4))
        area4 = np.sqrt(u4 * (u4 - a4) * (u4 - b4) * (u4 - b1))
        targetarea = area1 + area2 + area3 + area4
        
        assert np.all(targetarea > THRESHOLD_AREA)
        #use np.where!!!!!!!!!
        # This if statement may be redundant and thus unnecessary
        if(any(targetarea < AREA_BOUNDARY) and any(targetarea > THRESHOLD_AREA)):
            print("The target is on GFA", int(gfaid / 4))  
            newtargetindices = np.where(targetarea < AREA_BOUNDARY)
            targetindices.append(newtargetindices[0])
    """targetindices is an array of targets with their respective indices in the 
    RA and Dec list passed in that fall on certain GFAs denoted by the index 
    in the targetindices array."""
    return targetindices
        
        
#is_on_gfa(23658, 6.644525362152656, -9.055425745149217)
import desimodel.focalplane
#test point: (x,y) = (119, -380)
#test tileid = 23658
telra1, teldec1 = desimodel.footprint.get_tile_radec(23658)
testra, testdec = desimodel.focalplane.xy2radec(telra1, teldec1, -335, -210)
#is_on_gfa(23658, testra, testdec)

#print(testra, testdec)
testra2 = [testra, 6.644525362152656, 7]
testdec2 = [testdec, -9.055425745149217, -10]
is_on_gfa(23658, testra2, testdec2)

gfatable

7.11 -10.53
The target is on GFA 0
The target is on GFA 8


PETAL,CORNER,X,Y,Z,Q,RADIUS_DEG,RADIUS_MM
,,mm,mm,mm,degrees,,
int64,int64,float64,float64,float64,float64,float64,float64
0,0,116.279135121,-372.885546514,-17.053,287.31935812,1.54426530071,390.595017971
0,1,121.005023896,-387.4297715,-18.487,287.345026987,1.5999816328,405.886737469
0,2,91.784287159,-396.910065893,-18.631,283.02060759,1.60541232205,407.384285137
0,3,87.0587074141,-382.364889847,-17.198,282.826719301,1.54995479057,392.150643917
1,0,313.248421438,-233.323583313,-17.053,323.31935812,1.54426530071,390.595017971
1,1,325.620626724,-242.312300771,-18.487,323.345026987,1.5999816328,405.886737469
1,2,307.552931346,-267.157538162,-18.631,319.02060759,1.60541232205,407.384285137
1,3,295.180417053,-258.167869637,-17.198,318.826719301,1.54995479057,392.150643917


In [239]:
import matplotlib.pyplot as plt
#telra and teldec = 7.11 -10.53
"""mindec = -9.8
maxdec = -9.6
minra = 6.5
maxra = 8.47"""
mindec = -12.5
maxdec = -7
minra = 5.4
maxra = 8.9
ra = []
dec = []
import desimodel.focalplane
import desimodel.footprint

while mindec < maxdec:
    startra = minra
    while startra < maxra:
        ra.append(startra)
        dec.append(mindec)
        startra += .05
    mindec += .05

import desimodel.io
tiles = desimodel.io.load_tiles()
ra = np.asarray(ra)
dec = np.asarray(dec)

"""index = 0
for specific_tile in tiles:
    if(tiles[index]['TILEID'] == 23658):
        print(index)
    index += 1
#INDEX IS TILES[8254]"""
points = desimodel.footprint.find_points_in_tiles(tiles[8254], ra, dec)
"""for lists in points:
    if lists:
        print(lists)"""
#print(points)
new_ra = []
new_dec = []
for point_indices in points:
    plt.plot(ra[point_indices], dec[point_indices], 'b.')
    new_ra.append(ra[point_indices])
    new_dec.append(dec[point_indices])
targetindices = is_on_gfa(23658, new_ra, new_dec)
for gfaid in targetindices:
    for indices in gfaid:
        plt.plot(new_ra[indices], new_dec[indices], 'g.')
"""ra.append(8.452)
dec.append(-9.6977)
ra.append(6.644525362152656)
dec.append(-9.055)"""

"""targetindices = is_on_gfa(23658, ra, dec)
print(targetindices)
#print(tiles.dtype.names)
plt.plot(ra, dec, 'b.')
for gfaid in targetindices:
    for indices in gfaid:
        plt.plot(ra[indices], dec[indices], 'g.')
        #ra.remove(ra[indices])
        #dec.remove(dec[indices])"""
#savefig('entire_gfa_tolerance_100.jpg')

7.11 -10.53
The target is on GFA 0
The target is on GFA 1
The target is on GFA 2
The target is on GFA 3
The target is on GFA 4
The target is on GFA 5
The target is on GFA 6
The target is on GFA 7
The target is on GFA 8
The target is on GFA 9


"targetindices = is_on_gfa(23658, ra, dec)\nprint(targetindices)\n#print(tiles.dtype.names)\nplt.plot(ra, dec, 'b.')\nfor gfaid in targetindices:\n    for indices in gfaid:\n        plt.plot(ra[indices], dec[indices], 'g.')\n        #ra.remove(ra[indices])\n        #dec.remove(dec[indices])"